# Interpolation of FKtable on new x-grid

In [1]:
from validphys.coredata import FKTableData
from validphys.api import API
from validphys.fkparser import load_fktable
import numpy as np
import pandas as pd
from scipy.interpolate import (interp1d,RegularGridInterpolator, 
                               LinearNDInterpolator, interp2d, griddata,
                              NearestNDInterpolator, interpn)
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import warnings
# interp2d is deprecated and scipy complains
warnings.filterwarnings("ignore")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


### Simple Example of Interpolation using scipy.interpolate

In [2]:
x = np.array([1,2,3])
y = x
# default `kind` for spline interpolator is linear
f = interp1d(x, y, fill_value="extrapolate")
x_new = np.linspace(0.1,1,10)

f(x_new) - x_new

array([-2.77555756e-17, -5.55111512e-17,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00])

In [3]:
inp = {
    "fit": "210713-n3fit-001",
    "dataset_inputs": {"from_": "fit"},
    "use_cuts": "internal",
    "theoryid": 200
}

inp_400 = {
    "fit": "210713-n3fit-001",
    "dataset_inputs": {"from_": "fit"},
    "use_cuts": "internal",
    "theoryid": 400
}

In [4]:
data = API.data(**inp)
data_400 = API.data(**inp_400)

In [5]:
datasets = data.datasets
datasets_400 = data_400.datasets

fktable_dis = load_fktable(datasets[0].fkspecs[0])
fktable_dis_400 = load_fktable(datasets_400[0].fkspecs[0])

fktable_had = load_fktable(datasets[22].fkspecs[0])
fktable_had_400 = load_fktable(datasets_400[22].fkspecs[0])

new_xgrid = fktable_dis_400.xgrid

## DIS FKTable interpolation

In [6]:
def interp1d_(fktable, fktable_400):
    """
    """
    
    xgrid = fktable.xgrid
    xgrid_new = fktable_400.xgrid

    dfs=[]
    for d, grp in fktable.sigma.groupby('data'): 

        x_vals = xgrid[grp.index.get_level_values('x')]

        interpolators = {col: interp1d(x_vals, grp[col].values, kind='slinear', fill_value=0, bounds_error=False)
                        for col in grp.columns}
        
        tmp_index = pd.MultiIndex.from_product([[d],range(len(xgrid_new))], names=['data','x'])

        d = dict()
        
        for col in grp.columns:
            d[f"{col}"] = interpolators[col](xgrid_new)

        tmp_df = pd.DataFrame(d,index=tmp_index)

        dfs.append(tmp_df)
    
    return pd.concat(dfs, axis=0)



## Test of DIS interpolation

In [7]:
import time

# probably need to ignore the broken datasets as in the imagepdf nb

for ds, ds_400 in zip(datasets, datasets_400):
    ds_name = str(ds)

    for fk, fk_400 in zip(ds.fkspecs, ds_400.fkspecs):
        tab=load_fktable(fk)
        tab_400=load_fktable(fk_400)        

        if not (tab.hadronic and tab_400.hadronic):
            
            print()
            print(f"Dataset = {ds_name}")
            t0=time.time()
            interp1d_(tab, tab_400)
            t1=time.time()
            print(f'time needed for griddata = {t1-t0}')
            print()


Dataset = NMCPD_dw_ite
time needed for griddata = 0.15586400032043457


Dataset = NMCPD_dw_ite
time needed for griddata = 0.15121102333068848


Dataset = NMC
time needed for griddata = 0.31128811836242676


Dataset = SLACP_dwsh
time needed for griddata = 0.11115813255310059


Dataset = SLACD_dw_ite
time needed for griddata = 0.10625815391540527


Dataset = BCDMSP_dwsh
time needed for griddata = 0.30986690521240234


Dataset = BCDMSD_dw_ite
time needed for griddata = 0.20801424980163574


Dataset = CHORUSNUPb_dw_ite
time needed for griddata = 0.6159191131591797


Dataset = CHORUSNBPb_dw_ite
time needed for griddata = 0.5928218364715576


Dataset = NTVNUDMNFe_dw_ite
time needed for griddata = 0.05926990509033203


Dataset = NTVNBDMNFe_dw_ite
time needed for griddata = 0.04531574249267578


Dataset = HERACOMBNCEM
time needed for griddata = 0.1915600299835205


Dataset = HERACOMBNCEP460
time needed for griddata = 0.2445387840270996


Dataset = HERACOMBNCEP575
time needed for griddata = 0.

KeyboardInterrupt: 

In [11]:
print(datasets[0])
dis_200 = load_fktable(datasets[0].fkspecs[1])
sigma_dis_200 = dis_200.sigma
# get only first data point
sigma_dis_200 = sigma_dis_200[sigma_dis_200.index.get_level_values('data')==31]
xgrid_dis_200 = dis_200.xgrid[sigma_dis_200.index.get_level_values('x')]

dis_400 = load_fktable(datasets_400[0].fkspecs[0])
sigma_dis_400 = dis_400.sigma
# get only first data point
sigma_dis_400 = sigma_dis_400[sigma_dis_400.index.get_level_values('data')==31]
xgrid_dis_400 = dis_400.xgrid[sigma_dis_400.index.get_level_values('x')]

# sigma_dis_interpolated = interp1d_(load_fktable(datasets[0].fkspecs[0]), load_fktable(datasets_400[0].fkspecs[0]))
sigma_dis_interpolated = interp1d_(load_fktable(datasets[0].fkspecs[0]), load_fktable(datasets_400[0].fkspecs[0]))
sigma_dis_interpolated = sigma_dis_interpolated[sigma_dis_interpolated.index.get_level_values('data')==31]

NMCPD_dw_ite


In [12]:
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=xgrid_dis_200, y=sigma_dis_200[1],
                    mode='lines+markers',
                    name='Theory 200 grid'))

fig.add_trace(go.Scatter(x=xgrid_dis_400,
                         y=sigma_dis_interpolated['1'],
                    mode='lines+markers',
                    name='Linear Spline Interpolation Th 200'))



fig.add_trace(go.Scatter(x=xgrid_dis_400, y=sigma_dis_400[1],
                    mode='lines+markers',
                    name='Theory 400 grid'))



# fig.add_trace(go.Scatter(x=full_df.x_new, y=full_df['1_new_quad'],
#                     mode='lines+markers',
#                     name='Quadratic Spline Interpolation'))

# fig.add_trace(go.Scatter(x=full_df.x_new, y=full_df['1_new_cube'],
#                     mode='lines+markers',
#                     name='Cubic Spline Interpolation'))

fig.show()

## Hadronic FKTable Interpolation 

### Example1: Using deprecated interp2d

### Simple Example of 2D interpolation:

Given 3 data points and for each of them a function evaluated on a 2x2 xgrid. The function we want to fit is: f(x1,x2,data)=x1+x2 + data


In [13]:
ex_xgrid = np.array([0.1,0.2])

ex_dict = {"data":[0,0,0,0,1,1,1,1,2,2,2,2],"x1":[0,0,1,1,0,0,1,1,0,0,1,1],"x2":[0,1,0,1,0,1,0,1,0,1,0,1],
          "col1":[0.2,0.3,0.3,0.4,1.2,1.3,1.3,1.4,2.2,2.3,2.3,2.4]}

ex_df = pd.DataFrame.from_dict(ex_dict)
ex_df.set_index(['data', 'x1', 'x2'], inplace=True)


ex_dfs = []

for d, grp in ex_df.groupby('data'):
    x1_vals = ex_xgrid[grp.index.get_level_values('x1')]
    x2_vals = ex_xgrid[grp.index.get_level_values('x2')]

    interpolator = interp2d(x1_vals,x2_vals, grp['col1'])

    # test the interpolator
    new_ex_xgrid = np.array([0.1,0.15,0.2])
    len_grid = len(new_ex_xgrid)
    
    interpolated_grid = interpolator(new_ex_xgrid,new_ex_xgrid)
    
    tmp_index = pd.MultiIndex.from_product([range(3),range(3)], names=['x1','x2'])
    tmp_df = pd.DataFrame({'col1':interpolated_grid.flatten()},index=tmp_index)
    
    ex_dfs.append(tmp_df)

        
pd.concat(ex_dfs, axis=0)

col1
x1 x2      
0  0   0.20
   1   0.25
   2   0.30
1  0   0.25
   1   0.30
   2   0.35
2  0   0.30
   1   0.35
   2   0.40
0  0   1.20
   1   1.25
   2   1.30
1  0   1.25
   1   1.30
   2   1.35
2  0   1.30
   1   1.35
   2   1.40
0  0   2.20
   1   2.25
   2   2.30
1  0   2.25
   1   2.30
   2   2.35
2  0   2.30
   1   2.35
   2   2.40

In [14]:


def interp2d_(fktable, fktable_400):
    """
    """
    
    xgrid = fktable.xgrid
    xgrid_new = fktable_400.xgrid

    dfs=[]
    for d, grp in fktable.sigma.groupby('data'): 

        x1_vals = xgrid[grp.index.get_level_values('x1')]
        x2_vals = xgrid[grp.index.get_level_values('x2')]

        interpolators = [interp2d(x1_vals,x2_vals, grp[col].values)
                        for col in grp.columns]

        tmp_index = pd.MultiIndex.from_product([[d],range(50),range(50)], names=['data','x1','x2'])

        d = dict()

        for col, interp in zip(grp.columns, interpolators):
            d[f"{col}"] = interp(xgrid_new, xgrid_new).flatten()

        tmp_df = pd.DataFrame(d,index=tmp_index)

        dfs.append(tmp_df)
    
    return pd.concat(dfs, axis=0)


### Example2 Using: griddata

#### Simple example

In [15]:
ex_xgrid = np.array([0.1,0.2])

ex_dict = {"data":[0,0,0,0,1,1,1,1,2,2,2,2],"x1":[0,0,1,1,0,0,1,1,0,0,1,1],"x2":[0,1,0,1,0,1,0,1,0,1,0,1],
          "col1":[0.2,0.3,0.3,0.4,1.2,1.3,1.3,1.4,2.2,2.3,2.3,2.4]}

ex_df = pd.DataFrame.from_dict(ex_dict)
ex_df.set_index(['data', 'x1', 'x2'], inplace=True)


ex_dfs = []

for d, grp in ex_df.groupby('data'):
    x1_vals = ex_xgrid[grp.index.get_level_values('x1')]
    x2_vals = ex_xgrid[grp.index.get_level_values('x2')]
    

    # test the interpolator
    new_ex_xgrid = np.array([0.05,0.15,0.25])
    len_grid = len(new_ex_xgrid)
    
    new_ex_xgrid_mesh = np.meshgrid(new_ex_xgrid, new_ex_xgrid)

    # Flatten the grid into two separate 1D arrays
    new_ex_xgrid_flat = np.ravel(new_ex_xgrid_mesh[0])
    new_ex_ygrid_flat = np.ravel(new_ex_xgrid_mesh[1])

    # Evaluate griddata at all combinations of new_ex_xgrid values
    interpolated_grid = griddata(points=(x1_vals, x2_vals), 
                            values=grp['col1'], 
                            xi=(new_ex_xgrid_flat, new_ex_ygrid_flat), method='linear', fill_value=0)


    
    tmp_index = pd.MultiIndex.from_product([range(len(new_ex_xgrid)),range(len(new_ex_xgrid))], names=['x1','x2'])
    tmp_df = pd.DataFrame({'col1':interpolated_grid},index=tmp_index)
    
    ex_dfs.append(tmp_df)

        
pd.concat(ex_dfs, axis=0)

col1
x1 x2      
0  0    0.0
   1    0.0
   2    0.0
1  0    0.0
   1    0.3
   2    0.0
2  0    0.0
   1    0.0
   2    0.0
0  0    0.0
   1    0.0
   2    0.0
1  0    0.0
   1    1.3
   2    0.0
2  0    0.0
   1    0.0
   2    0.0
0  0    0.0
   1    0.0
   2    0.0
1  0    0.0
   1    2.3
   2    0.0
2  0    0.0
   1    0.0
   2    0.0

In [16]:
# # possibility, but quite slow
# grid_z0 = griddata(points, values, (grid_x, grid_y), method='nearest')
# grid_z1 = griddata(points, values, (grid_x, grid_y), method='linear')

# fill_value = 123  # Whatever you like
# grid_z0[np.isnan(grid_z1)] = fill_value

In [26]:
def griddata_(fktable, fktable_400):
    """
    """
    
    xgrid = fktable.xgrid
    xgrid_new = fktable_400.xgrid

    dfs=[]
    for d, grp in fktable.sigma.groupby('data'): 

        x1_vals = xgrid[grp.index.get_level_values('x1')]
        x2_vals = xgrid[grp.index.get_level_values('x2')]

        new_xgrid_mesh = np.meshgrid(xgrid_new, xgrid_new)

        # Flatten the grid into two separate 1D arrays
        new_x1grid_flat = np.ravel(new_xgrid_mesh[0])
        new_x2grid_flat = np.ravel(new_xgrid_mesh[1])

#         interpolated_grids = {col: griddata(points=(x1_vals,x2_vals), values=grp[col].values,
#                                       xi=(new_x1grid_flat,new_x2grid_flat), method='nearest')
#                         for col in grp.columns}
        
        interpolated_grids = {col: griddata(points=(x1_vals,x2_vals), values=grp[col].values,
                              xi=(new_x1grid_flat,new_x2grid_flat), method='linear', fill_value=0)
                for col in grp.columns}

        tmp_index = pd.MultiIndex.from_product([[d],range(len(xgrid_new)),range(len(xgrid_new))], names=['data','x1','x2'])

        col_dict = dict()

        # set to zero in extrapolation regions
        idx_x1_m = np.where(new_x1grid_flat<np.min(x1_vals))[0]
        idx_x1_p = np.where(new_x1grid_flat>np.max(x1_vals))[0]
        idx_x1 = np.unique(np.concatenate((idx_x1_m, idx_x1_p)))
        
        idx_x2_m = np.where(new_x2grid_flat<np.min(x2_vals))[0]
        idx_x2_p = np.where(new_x2grid_flat>np.max(x2_vals))[0]
        idx_x2 = np.unique(np.concatenate((idx_x2_m, idx_x2_p)))
        
        extrapolation_region = np.unique(np.concatenate((idx_x1,idx_x2)))    

        for col in grp.columns:            
#             interpolated_grids[col][extrapolation_region]=0
            col_dict[f"{col}"] = interpolated_grids[col]
        
        tmp_df = pd.DataFrame(col_dict,index=tmp_index)

        dfs.append(tmp_df)

    return pd.concat(dfs, axis=0)

## Test of Hadronic Implementation

In [27]:
ds=20
dp=4

had_200 = load_fktable(datasets[ds].fkspecs[0])
sigma_had_200 = had_200.sigma
# get only 1 datapoint
sigma_had_200 = sigma_had_200[sigma_had_200.index.get_level_values('data')==dp]
# fixed x1, sigma as func of x2
sigma_had_200 = sigma_had_200[sigma_had_200.index.get_level_values('x1')==sigma_had_200.index.get_level_values('x1')[33]]

x2grid_had_200 = had_200.xgrid[sigma_had_200.index.get_level_values('x2')]
x1val_200 = np.unique(had_200.xgrid[sigma_had_200.index.get_level_values('x1')])

sigma_had_200

had_400 = load_fktable(datasets_400[ds].fkspecs[0])
sigma_had_400 = had_400.sigma
# get only 1 datapoint
sigma_had_400 = sigma_had_400[sigma_had_400.index.get_level_values('data')==dp]
# fixed x1, sigma as func of x2
x1_idx = np.where(np.abs(had_400.xgrid - x1val_200) == np.min(np.abs(had_400.xgrid - x1val_200)) )[0][0]
sigma_had_400 = sigma_had_400[sigma_had_400.index.get_level_values('x1')==x1_idx]

x2grid_had_400 = had_400.xgrid[sigma_had_400.index.get_level_values('x2')]

sigma_had_interpolated = griddata_(load_fktable(datasets[ds].fkspecs[0]), load_fktable(datasets_400[ds].fkspecs[0]))
# get only 1 datapoint
sigma_had_interpolated = sigma_had_interpolated[sigma_had_interpolated.index.get_level_values('data')==dp]
# fixed x1, sigma as func of x2
sigma_had_interpolated = sigma_had_interpolated[sigma_had_interpolated.index.get_level_values('x1')==x1_idx]


In [28]:
fig = go.Figure()
col=15
# Add traces
fig.add_trace(go.Scatter(x=x2grid_had_200, y=sigma_had_200[col],
                    mode='lines+markers',
                    name='Theory 200 grid'))


fig.add_trace(go.Scatter(x=x2grid_had_400, y=sigma_had_400[col],
                    mode='lines+markers',
                    name='Theory 400 grid'))

fig.add_trace(go.Scatter(x=x2grid_had_400,
                         y=sigma_had_interpolated[f'{col}'],
                    mode='lines+markers',
                    name='2D Interpolation Th 200'))



fig.show()

In [72]:
sigma_had_200[sigma_had_200.index.get_level_values('x1')==3]

15            16            17            19            20   \
data x1 x2                                                                     
5    3  5   0.000031 -3.571636e-08 -2.347446e-22 -2.464362e-22  0.000000e+00   
        6   0.000471 -9.049938e-07 -7.402461e-21 -3.793357e-21 -4.042137e-26   
        7   0.000361 -1.823013e-06  3.694787e-22 -4.686687e-21  0.000000e+00   
        8   0.000001 -3.412575e-07  6.821376e-21  1.659741e-21  4.718722e-22   
        9  -0.000004  1.404160e-06 -2.003831e-21 -2.200502e-22  4.503577e-22   
        10  0.000034  1.860175e-06 -2.269753e-21  6.673897e-23  1.487929e-22   
        11  0.000019  2.062199e-06 -1.659004e-21 -2.123182e-22  5.162952e-22   
        12  0.000011  2.294315e-06  1.312326e-22 -2.564064e-22 -2.434220e-22   
        13  0.000010  2.411945e-06  9.953018e-22 -3.502632e-22  1.070315e-22   
        14  0.000008  2.457784e-06  1.640672e-21  1.246212e-22  1.446957e-22   
        15  0.000007  2.470180e-06  1.374566e-21 -1.690613e-23 -2.714812e-22   
        16  0.000006  2.461037e-06  1.027197e-21 -1.211539e-22 -1.473655e-23   
        17  0.000005  2.433742e-06  5.390216e-22  2.049448e-22 -3.450799e-23   
        18  0.000005  2.391082e-06 -3.341502e-22 -5.696743e-24  5.996929e-24   
        19  0.000004  2.343491e-06  3.771601e-22  1.616341e-22 -9.183716e-24   
        20  0.000004  2.294769e-06 -1.875424e-22  1.857198e-22  1.169091e-23   
        21  0.000003  2.242866e-06 -3.543874e-22  8.512086e-23 -1.844024e-23   
        22  0.000003  2.192014e-06  2.456437e-22  2.226656e-22  3.207217e-22   
        23  0.000003  2.140745e-06 -4.973361e-22  1.462274e-23 -5.643607e-23   
        24  0.000003  2.088470e-06 -2.708403e-23  9.058139e-23  1.445176e-23   
        25  0.000003  2.038251e-06 -2.406535e-22  2.982936e-23  1.802947e-23   
        26  0.000002  1.990294e-06 -1.007321e-22 -1.423298e-22 -3.406102e-24   
        27  0.000002  1.943447e-06  1.988325e-22  1.958416e-23  1.820095e-24   
        28  0.000002  1.898113e-06 -4.046807e-22  7.766506e-23  3.051080e-24   
        29  0.000002  1.858890e-06 -3.159037e-23  7.838814e-23  1.117154e-23   
        30  0.000002  1.817126e-06  4.861619e-22  2.884338e-23 -7.344062e-24   
        31  0.000002  1.773098e-06 -3.028709e-22  6.827333e-23  6.849051e-23   
        32  0.000002  1.733839e-06  6.362701e-23  9.819492e-23  3.895716e-23   
        33  0.000002  1.697790e-06  2.457424e-22  9.058339e-23  2.210077e-22   
        34  0.000002  1.660911e-06 -7.596831e-23  4.576457e-23  9.985430e-24   
        35  0.000001  1.626865e-06  5.732778e-23  3.175686e-23  3.575086e-25   
        36  0.000001  1.593929e-06 -1.461566e-22  2.109933e-23  2.805942e-24   
        37  0.000001  1.592943e-06  4.467603e-23 -5.208351e-23 -1.071359e-23   
        38  0.000001  1.531824e-06  3.511682e-22  1.625476e-23  9.194051e-24   
        39  0.000002  2.446829e-06 -1.056329e-22 -4.170457e-23  6.260293e-23   

                     24            25            29            30   \
data x1 x2                                                           
5    3  5   6.229416e-06 -6.229416e-06 -6.060400e-08  6.522245e-11   
        6   9.410777e-05 -9.410776e-05 -9.666424e-07  1.342987e-09   
        7   7.223379e-05 -7.223378e-05 -8.003624e-07  1.619324e-09   
        8   2.444969e-07 -2.444943e-07 -3.747381e-08 -2.829476e-09   
        9  -8.558997e-07  8.558993e-07 -2.657237e-09 -7.044804e-09   
        10  6.787464e-06 -6.787463e-06 -7.964934e-08 -8.238936e-09   
        11  3.846456e-06 -3.846458e-06 -4.752754e-08 -8.802672e-09   
        12  2.155623e-06 -2.155623e-06 -2.744046e-08 -9.348312e-09   
        13  1.953358e-06 -1.953359e-06 -2.381107e-08 -9.576574e-09   
        14  1.681613e-06 -1.681611e-06 -2.000669e-08 -9.594939e-09   
        15  1.401205e-06 -1.401207e-06 -1.634923e-08 -9.515576e-09   
        16  1.221484e-06 -1.221484e-06 -1.395098e-08 -9.372130e-09   
        17  1.067757e-06 -1.067757e-06 -1.194810e-08 -9.182985e-09   
      

## Test griddata versus interp2d

In [63]:
sigma_1 = interp2d_(fktable_had, fktable_had_400)
sigma_2 = griddata_(fktable_had, fktable_had_400)

In [64]:
import time

# probably need to ignore the broken datasets as in the imagepdf nb

for ds, ds_400 in zip(datasets, datasets_400):
    ds_name = str(ds)

    for fk, fk_400 in zip(ds.fkspecs, ds_400.fkspecs):
        tab=load_fktable(fk)
        tab_400=load_fktable(fk_400)        

        if tab.hadronic and tab_400.hadronic:
            
            print()
            print(f"Dataset = {ds_name}")
            t0=time.time()
            griddata_(tab, tab_400)
            t1=time.time()
            print(f'time needed for griddata = {t1-t0}')
            print()
#             t0=time.time()
#             interp2d_(tab, tab_400)
#             t1=time.time()
#             print(f'time needed for interp2d = {t1-t0}')
            
            


Dataset = DYE886R_dw_ite
time needed for griddata = 2.8714170455932617


Dataset = DYE886R_dw_ite
time needed for griddata = 3.4969239234924316


Dataset = DYE886P
time needed for griddata = 34.607521057128906


Dataset = DYE605_dw_ite
time needed for griddata = 41.15037703514099


Dataset = DYE906R_dw_ite
time needed for griddata = 1.9671008586883545


Dataset = DYE906R_dw_ite
time needed for griddata = 1.9105918407440186


Dataset = DYE906R_dw_ite
time needed for griddata = 1.7240381240844727


Dataset = DYE906R_dw_ite


QhullError: QH6154 Qhull precision error: Initial simplex is flat (facet 1 is coplanar with the interior point)

While executing:  | qhull d Qz Q12 Qbb Qt Qc
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 1095309642  delaunay  Qz-infinity-point  Q12-allow-wide  Qbbound-last
  Qtriangulate  Qcoplanar-keep  _pre-merge  _zero-centrum  Qinterior-keep
  Pgood  _max-width 0.46  Error-roundoff 1.3e-15  _one-merge 9e-15
  Visible-distance 2.6e-15  U-max-coplanar 2.6e-15  Width-outside 5.2e-15
  _wide-facet 1.5e-14  _maxoutside 1e-14

The input to qhull appears to be less than 3 dimensional, or a
computation has overflowed.

Qhull could not construct a clearly convex simplex from points:
- p1(v4):  0.49  0.93 0.022
- p22(v3):   0.7  0.93  0.93
- p21(v2):  0.93  0.93  0.73
- p0(v1):  0.47  0.93     0

The center point is coplanar with a facet, or a vertex is coplanar
with a neighboring facet.  The maximum round off error for
computing distances is 1.3e-15.  The center point, facets and

In [29]:
np.max(sigma_1.to_numpy()- sigma_2.to_numpy())

0.00019656477386740742